<a href="https://colab.research.google.com/github/SelinaZ07/SelinaZ07/blob/main/Canva_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Few thins that needs to be imported before using this template is(recommended to to all the files into a google folde): The three font files we want, the certificate template (no words pre-filled), the certificate data folder.

In [ ]:
#@title [TODO] [REPLACE - Doesn't work] Generate certificates for selected participants on Canva

!pip install pillow pandas
from PIL import Image, ImageDraw, ImageFont
import os
import pandas as pd
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.auth import default

# Authenticate and initialize the Drive API
auth.authenticate_user()
creds, _ = default()
drive_service = build('drive', 'v3', credentials=creds)

# Create subfolders in Google Drive folder
def get_or_create_folder(service, name, parent_id):
    query = f"name='{name}' and mimeType='application/vnd.google-apps.folder' and '{parent_id}' in parents"
    results = service.files().list(q=query, fields="files(id, name)").execute()
    folders = results.get('files', [])
    if folders:
        return folders[0]['id']
    folder_metadata = {
        'name': name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [parent_id]
    }
    folder = service.files().create(body=folder_metadata, fields='id').execute()
    return folder['id']

def upload_to_drive(local_path, file_name, mime_type, folder_id):
    file_metadata = {'name': file_name, 'parents': [folder_id]}
    media = MediaFileUpload(local_path, mimetype=mime_type)
    uploaded = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    return uploaded.get('id')

# Customization (no need for any changes)
canva_color = (0, 191, 99)
text_color = (255, 255, 255)

# get the Canva template
template_path = "The Template.png" # @param {"type":"string"}
output_folder = "Canva Project"
os.makedirs(output_folder, exist_ok=True)

# Font paths and sizes
name_font_path = "GreatVibes-Regular.ttf" # @param {"type":"string"}
cert_id_font_path = "HammersmithOne-Regular.ttf" # @param {"type":"string"}
cert_url_font_path = "LibreBaskerville-Regular.ttf" # @param {"type":"string"}

name_font = ImageFont.truetype(name_font_path, 140)
cert_id_font = ImageFont.truetype(cert_id_font_path, 35)
cert_url_font = ImageFont.truetype(cert_url_font_path, 20)

# Link to google drive folder and create subfolders in it
folder_id = "1m5mUtFVHVDuAHQR_-se9cZyYJ_oCA89g" # @param {"type":"string"}
png_folder_id = get_or_create_folder(drive_service, "png", folder_id)
pdf_folder_id = get_or_create_folder(drive_service, "pdf", folder_id)

# create and upload certificates
for i, row in df.iterrows():
    full_name = row["Full Name"]
    certificate_id = row["Certificate ID"]
    certificate_url = row["Certificate URL"]

    img = Image.open(template_path)
    draw = ImageDraw.Draw(img)

    # Draw name
    name_bbox = draw.textbbox((0, 0), full_name, font=name_font)
    text_width = name_bbox[2] - name_bbox[0]
    text_height = name_bbox[3] - name_bbox[1]
    image_width, image_height = img.size
    x_position = (image_width - text_width) / 2
    y_position = image_height / 2.1
    draw.text((x_position, y_position), full_name, font=name_font, fill=canva_color)

    # Draw certificate ID
    cert_text = f"Certificate ID: {certificate_id}"
    cert_bbox = draw.textbbox((0, 0), cert_text, font=cert_id_font)
    cert_x = (image_width - (cert_bbox[2] - cert_bbox[0])) / 2
    cert_y = y_position + text_height + 60
    draw.text((cert_x, cert_y), cert_text, font=cert_id_font, fill=canva_color)

    # Draw certificate URL
    url_bbox = draw.textbbox((0, 0), certificate_url, font=cert_id_font)
    url_x = (image_width - (url_bbox[2] - url_bbox[0])) / 2 + 380
    url_y = image_height - (url_bbox[3] - url_bbox[1]) - 9.2
    draw.text((url_x, url_y), certificate_url, font=cert_url_font, fill=text_color)

    # Save PNG and PDF
    png_path = os.path.join(output_folder, f"{full_name}.png")
    pdf_path = os.path.join(output_folder, f"{full_name}.pdf")
    img.save(png_path, "PNG")
    img.save(pdf_path, "PDF", resolution=100.0)

    # Upload to Google Drive
    upload_to_drive(png_path, f"{full_name}.png", "image/png", png_folder_id)
    print(f"Uploaded {full_name}.png to PNG folder.")

    upload_to_drive(pdf_path, f"{full_name}.pdf", "application/pdf", pdf_folder_id)
    print(f"Uploaded {full_name}.pdf to PDF folder.")